In [ ]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file"{name}" with length {length} bytes'.format(name = fn, length = len(uploaded[fn])))

!mkdir -p ~/.kaggle/
!mv kaggle.json ~/.kaggle/ 
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions list

In [ ]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

In [ ]:
import os
os.getcwd()

In [ ]:
!unzip house-prices-advanced-regression-techniques

In [ ]:
import pandas as pd 
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.shape, test.shape

In [ ]:
train.info()

In [ ]:
test.info()

### 1. 이상치 제거
- 이상치 : 적정하지 않게 측정된 데이터
  - 예를 들면 집 상태가 좋지 않은 데 집값이 비싸다든가?
- 주관성이 반영된 데이터는 적절히 삭제해줄 필요가 있다.
  - ex) OverallCond, OverallQual : 측정자의 주관이 반영될 여지가 있음
    --> 제거해줘도 ㅇㅋ

In [ ]:
train.drop(train[(train['OverallQual'] < 4 ) & (train['SalePrice'] > 200000)].index, inplace = True)
train.drop(train[(train['OverallCond'] < 4 ) & (train['SalePrice'] > 200000)].index, inplace = True)
train.reset_index(drop = True, inplace = True)
print(train.shape)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm

(mu, sigma) = norm.fit(train['SalePrice']) # 확률밀도 함수 Normalize

fig, ax = plt.subplots(figsize=(10, 6))
sns.histplot(train['SalePrice'], color = 'b', stat = 'probability')
ax.xaxis.grid(False)
ax.set(ylabel = 'frequency')
ax.set(xlabel = 'SalePrice')
ax.set(title = "SalePrice Distribution")

plt.axvline(mu, color = 'r', linestyle='--')
plt.text(mu + 10000, 0.11, 'Mean of SalePrice', rotation = 0 , color = 'r')
fig.show()

#### 위 그래프는 정규분포를 만족하지 않음 : 선형회귀는 정규 분포를 만족해야 하기 때문에 SalePrice를 변환해준다

In [ ]:
import numpy as np

train['SalePrice'] = np.log1p(train['SalePrice'])

(mu, sigma) = norm.fit(train['SalePrice']) # 확률밀도 함수 Normalize

fig, ax = plt.subplots(figsize=(10, 6))
sns.histplot(train['SalePrice'], color = 'b', stat = 'probability')
ax.xaxis.grid(False)
ax.set(ylabel = 'frequency')
ax.set(xlabel = 'SalePrice')
ax.set(title = "SalePrice Distribution")

plt.axvline(mu, color = 'r', linestyle='--')
plt.text(mu + 0.05, 0.111, 'Mean of SalePrice', rotation = 0 , color = 'r')
plt.ylim(0, 0.12)
fig.show()

In [ ]:
train_ID = train['Id']
test_ID = test['Id']
train.drop(['Id'], axis = 1, inplace = True)
test.drop(['Id'], axis = 1, inplace = True)
train.shape, test.shape

In [ ]:
y = train['SalePrice'].reset_index(drop = True)
train = train.drop('SalePrice', axis = 1)
train.shape, test.shape, y.shape

In [ ]:
# df 합침
all_df = pd.concat([train, test]).reset_index(drop = True)
all_df.shape

### 결측치 처리

In [ ]:
def check_na(data, head_num = 5):
   
  isnull_na = (data.isnull().sum() / len(data)) * 100 # 결측치 데이터 / 전체 데이터 * 100 값을 모든 feature에 대해 저장함
  data_na = isnull_na.drop(isnull_na[isnull_na == 0].index).sort_values(ascending = False) # feature = 0
  missing_data = pd.DataFrame({"Missing Ratio" : data_na,
                               "Data Type" : data.dtypes[data_na.index]})
  print("결측치 데이터 칼럼과 건수 : \n", missing_data.head(head_num))

check_na(all_df, 20)

In [ ]:
# 위 함수 뜯어보기
all_df.isnull().sum() # 각 feature 별로 null 값이 몇 개인지 보여줌. 이거 자체도 df의 모양임
                        # 그렇기 때문에 아래의 data_na 값은 결측치가 없는 feature는 전부 탈락시키고, 
                        # 내림차순으로 정렬됨
                        # 그 다음부터는 새로운 df를 만들어서 결측치 비율을 보여주는 거니까 어렵지 않을 것

#### 결측치 처리 방법은 다양하지만, 책에서는 결측치가 높은 상위 6개의 특성을 제거하는 식으로 진행했음

In [ ]:
all_df.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'LotFrontage'],
            axis = 1,
            inplace = True)
check_na(all_df, 20)

- 결측치를 채우는 방법은 2가지가 있다.
1. 각 변수를 확인, 개별적으로 필요한 데이터를 채워 넣는다
  - 문자 데이터는 빈도수 위주, 수치 데이터는 평균 or 중위값으로 넣음

2. 한꺼번에 0 혹은 None 값을 채워넣는다(???)

In [ ]:
import numpy as np

cat_all_vars = train.select_dtypes(exclude = [np.number]) # 숫자형 데이터 빼고 모든 데이터 타입에 해당하는 데이터들을 선택
print("가공 전 : " ,len(list(cat_all_vars)))
final_cat_vars = []
for v in cat_all_vars:
  if v not in ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'LotFrontage']:
    final_cat_vars.append(v)
print("가공 후 : ", len(final_cat_vars))

In [ ]:
final_cat_vars # final_cat_var는 column_name의 list이다

In [ ]:
for i in final_cat_vars:
  all_df[i] = all_df[i].fillna(all_df[i].mode()[0]) # mode : 최빈값(가장 잦은 빈도로 나오는 값)
check_na(all_df, 20)

In [ ]:
all_df[final_cat_vars].mode() # 왜 하나만 뜨는거임? - 그게 '가장 자주' 나오는 값이니까..!

In [ ]:
# 수치형 데이터 처리
num_all_vars = list(train.select_dtypes(include = [np.number]))
print("all_vars : ", len(num_all_vars))
num_all_vars.remove('LotFrontage')

print("final_cat_vars : ", len(num_all_vars))
for i in num_all_vars:
  all_df[i].fillna(value = all_df[i].median(), inplace = True)

check_na(all_df, 20) # 결측치가 하나도 없기 때문에 다 사라진 것을 확인할 수 있다

### 왜도(Skewness) 값 처리
- 왜도 : 분포의 비대칭도. 즉 정규분포 같이 대칭인 분포는 왜도가 0
- 모델의 신뢰성에 영향을 미친다

In [ ]:
from scipy.stats import skew

def find_skew(x):
  return skew(x)

skew_features = all_df[num_all_vars].apply(find_skew).sort_values(ascending=False)
skew_features

In [ ]:
# 시각화 (변수 LotArea 값이 너무 커서 다른 변수들의 분포도를 확인하기 어려워 제거함)
skewnewss_index = list(skew_features.index)
skewnewss_index.remove('LotArea')
all_numeric_df = all_df.loc[:, skewnewss_index]

fig, ax = plt.subplots(figsize = (10, 6))
ax.set_xlim(0, all_numeric_df.max().sort_values(ascending = False)[0])
ax = sns.boxplot(data = all_numeric_df[skewnewss_index], orient = 'h', palette = 'Set1')
ax.xaxis.grid(False)
ax.set(ylabel = 'feature names')
ax.set(xlabel = 'numeric values')
ax.set(title = "Numeric Distribution of Features Before Box-Cox Transformation")
sns.despine(trim = True, left = True)

- Boxplot을 그렸음에도 왜도가 높다면 Box가 잘 나타나지 않음
- 이런 경우 회귀값이 왜곡될 수 있기 때문에, Box-Cox Transformation을 시도한다

In [ ]:
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

high_skew = skew_features[skew_features > 1]
high_skew_index = high_skew.index
print("Transformation 전 : \n", all_df[high_skew_index].head())

for num_var in high_skew_index:
  all_df[num_var] = boxcox1p(all_df[num_var], boxcox_normmax(all_df[num_var] + 1))

print("Transformation 후 : \n", all_df[high_skew_index].head())

### 도출 변수
- 주어진 변수 내에서 새로운 변수를 도출하는 과정
- 차원 축소와는 다름 : 차원 축소는 PCA를 이용해 계산적으로 유용한 성분들을 뽑아내는 과정이라고 할 수 있음
- 반면 도출 변수는 여러 feature의 특징을 고려해 새로운 변수로 만들어내므로, 연구자의 주관적인 분석이 더 들어감
- 즉, 변수가 아무리 많더라도 여러 변수를 조합해 하나의 변수로 재그룹화 할 수 있다

In [ ]:
all_df['TotalSF'] = all_df['TotalBsmtSF'] + all_df["1stFlrSF"] + all_df['2ndFlrSF']
all_df.drop(['TotalBsmtSF', '1stFlrSF', '2ndFlrSF'], axis = 1)
print(all_df.shape)

- SF : Square Feet - 집의 층별 크기를 의미
  - 근데 집이 클수록 비싼게 통상적이라고 생각하면 변수를 통합해도 무방함

In [ ]:
# 유사하게 다른 변수들에도 적용할 수 있음
all_df['Total_Bathrooms'] = (all_df['FullBath'] + (0.5*all_df['HalfBath']) + all_df['BsmtFullBath'] + (0.5*all_df['BsmtHalfBath']))
all_df['Total_porch_sf'] = (all_df['OpenPorchSF'] + all_df['3SsnPorch'] + all_df['EnclosedPorch'] + all_df['ScreenPorch'])
all_df = all_df.drop(['FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath', 'OpenPorchSF', '3SsnPorch', 'EnclosedPorch',
                      'ScreenPorch'], axis = 1)
print(all_df.shape)

- 연도에 접근하기

In [ ]:
num_all_vars = list(train.select_dtypes(include = [np.number]))
year_feature = []
for var in num_all_vars:
  # 정규식까지 가지 않더라도 string 내에서도 in을 이용해 원하는 문자열에 함수를 적용할 수 있다
  if 'Yr' in var:
    year_feature.append(var)
  elif 'Year' in var:
    year_feature.append(var)
  else:
    print(var, "is not related with Year")
print(year_feature)

- 시각화

In [ ]:
fig, ax = plt.subplots(3, 1, figsize = (10, 6), sharex = True, sharey = True)
for i, var in enumerate(year_feature):
  if var != 'YrSold':
    ax[i].scatter(train[var], y, alpha = 0.3)
    ax[i].set_title('{}'.format(var), size = 15)
    ax[i].set_ylabel('SalePrice', size = 15, labelpad= 12.5) # 위에서 SalePrice는 로그 스케일로 바꿨음
  
plt.tight_layout() # 각 그래프들의 구성 요소가 겹치지 않게끔 자동으로 조절해주는 역할인 거 같음
plt.show()

- 리모델링을 하면 호가가 높아지니 리모델링만 기준으로 하고 나머지 변수는 삭제함 
- 여기서 통계적으로 유사한 변수들을 모아 상관관계를 분석하는 것도 도움이 된다.

In [ ]:
all_df = all_df.drop(['YearBuilt', 'GarageYrBlt'], axis = 1)
print(all_df.shape)

- 변수 추가 : 팔린 날짜와 리모델링한 날짜 사이의 변수를 추가하고 이를 가격과 함께 시각화해본다

In [ ]:
YearsSinceRemodel = train['YrSold'].astype(int) - train['YearRemodAdd'].astype(int)

fig, ax = plt.subplots(figsize = (10, 6))
ax.scatter(YearsSinceRemodel, y, alpha = 0.3)
fig.show()

In [ ]:
all_df['YearsSinceRemodel'] = all_df['YrSold'].astype(int) - all_df['YearRemodAdd'].astype(int)
all_df = all_df.drop(['YrSold', 'YearRemodAdd'], axis = 1)
print(all_df.shape)

### 더미 변수 응용하기
- 더미 변환 : ex) 성별을 1과 2로 바꾸는 과정

In [ ]:
all_df['PoolArea'].value_counts()

In [ ]:
# 집의 수영장 크기를 의미하는 듯 한데, 이를 단순화한다 : 있다 / 없다로
def count_dummy(x):
  if x > 0 :
    return 1
  else:
    return 0

all_df['PoolArea'] = all_df['PoolArea'].apply(count_dummy)
all_df['PoolArea'].value_counts()

In [ ]:
all_df['GarageArea'] = all_df['GarageArea'].apply(count_dummy
)
all_df['GarageArea'].value_counts()

In [ ]:
all_df['Fireplaces'] = all_df['Fireplaces'].apply(count_dummy)
all_df['Fireplaces'].value_counts()

--------------------------------------------------------------------
### 데이터와 상관없는 시간 : Label / Oridnal / One-Hot Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder

In [ ]:
temp = pd.DataFrame({'Food_Name' : ['Apple', 'Chicken', 'Broccoli'],
                     'Calories' : [95, 231, 50]})


In [ ]:
# 1. LabelEncoder - 정수로 출력
encoder = LabelEncoder()
encoder.fit(temp['Food_Name'])
labels = encoder.transform(temp['Food_Name'])
print(list(temp['Food_Name']), labels)

In [ ]:
# 2. OrdinalEncoder - 2차원 input / 각 행으로 출력됨
encoder = OrdinalEncoder()
labels = encoder.fit_transform(temp[['Food_Name']]) 
print(labels) 

In [ ]:
# 3. OneHotEncoder
# sklearn에서 onehot encoder를 지원하지만, 텍스트를 Label Encoding을 먼저 사용해 바꾼 뒤 reshape를 써야 함

# 다행히 pandas에서 get_dummies를 지원한다. - 대신 원래의 column이 바뀌어 버리는 문제가 있음. 
# temp = pd.get_dummies(temp)
# print(temp)
# print(temp.shape)

In [ ]:
temp

- 서열 척도(등급 등)은 ordinalencoder를 쓰는 게 더 합리적일 수 있으나, 그렇다고 숫자에 의미가 있다고 데이터가 여기지는 않는다.
- 따라서 필자는 Pandas를 활용해 직접 바꾸는 게 더낫다고 생각함(가독성의 측면에서)

In [ ]:
temp['Food_No'] = temp.Food_Name.replace(to_replace=['Chicken', 'Broccoli', 'Apple'], value = [1,2,3])
print(temp[['Food_Name', 'Food_No']])

- 뭘 쓰느냐에 따라 유용한 알고리즘이 다르긴 하다
  - Ordinal : NonTree에서 더 유용(회귀, SVM 등)
  - OneHot : Tree 기반에서 더 유용(DT, RF)
- 근데 어차피 각 모델별로 다 돌릴거잖아?

------------------------------------

In [ ]:
# 본 데이터 : 모든 것에 대해 one-hot encoding을 돌린다
all_df = pd.get_dummies(all_df).reset_index(drop = True)
all_df.shape

### Ml 모형 학습 및 평가

In [ ]:
# 데이터 세트 재분리
X = all_df.iloc[:len(y), :]
X_test = all_df.iloc[len(y):, :]
X.shape, y.shape, X_test.shape

In [ ]:
# 훈련 , 테스트 세트 분리
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
# 혹은 CV
from sklearn.model_selection import KFold

cv = KFold(n_splits = 5, random_state = 42, shuffle = True)

#### 회귀 모델 평가 지표 : 오차(Error)
1. 절댓값 오차의 평균 = Mean Absolute Error
2. 제곱 오차 = Mean Squared Error

In [ ]:
# 1. MAE

def mean_absolute_error(y_true, y_pred):
  error = 0
  for yt, yp in zip(y_true, y_pred):
    error = error + np.abs(yt - yp)
  mae = error / len(y_true)

  return mae

# 2. MSE

def mean_squared_error(y_true, y_pred):

  error = 0 
  for yt, yp in zip(y_true, y_pred):
    error = error + (yt - yp) ** 2
  mse = error / len(y_true)

  return mse

# 3. RMSE

def root_rmse_squared_error(y_true, y_pred):

  error = 0

  for yt, yp in zip(y_true, y_pred):
    error = error + (yt - yp) ** 2

  mse = error / len(y_true)
  rmse = np.round(np.sqrt(mse), 3)

  return rmse

In [ ]:
y_true = [400, 300, 800]
y_pred = [380, 320, 777]

print(mean_absolute_error(y_true, y_pred), mean_squared_error(y_true, y_pred), root_rmse_squared_error(y_true, y_pred))


#### 왜 일반적으로 rmse를 더 많이 이용할까?
- 오차가 커지면 rmse의 상승폭이 mae에 비해 엄청 커짐
  - 즉 rmse가 오차 증가를 더 직관적으로 확인할 수 있는 지표이기 때문이다.

#### 예측의 정확도 판단하기
- 결정계수(Coefficient of Determination) 이용, R^2도 같은 말
  - 이는 분산 기반으로 예측 성능을 평가하며,
  1 - ( 예측값의 분산 / 실제값의 분산) 으로 평가됨
  - 참고로 분산은 편차(각 값 - 평균 값)의 제곱합으로 계산됨

In [ ]:
# rmse는 사이킷런에 없다 : 직접 만들어 쓰자 - mse는 있으니까
from sklearn.metrics import mean_squared_error

def rmsle(y_true, y_pred):
  return np.sqrt(mean_squared_error(y_true, y_pred))

#### 모형 정의부터 학습, 평가까지 진행

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression

def cv_rmse(model, n_folds = 5):
  cv = KFold(n_splits = n_folds, random_state = 42, shuffle = True)
  rmse_list = np.sqrt(-cross_val_score(model, X, y, scoring = 'neg_mean_squared_error', cv = cv, # 이거 왜 -지?
                                       error_score = 'raise'))
  print("CV RMSE VALUE LIST : ", np.round(rmse_list, 4))
  print("CV RMSE mean value : ", np.round(np.mean(rmse_list), 4))
  return (rmse_list)

n_folds = 5
rmse_scores = {}
lr_model = LinearRegression()

score = cv_rmse(lr_model, n_folds) 
print(f"LR - mean : {score.mean()}, std : {score.std()}")
rmse_scores['linear regression'] = (score.mean(), score.std())

# 뭔가 표준화가 안 된 느낌이지만 그냥 넘어간다

In [ ]:
from sklearn.model_selection import cross_val_predict

X = all_df.iloc[:len(y), :]
X_test = all_df.iloc[len(y): , :]
X.shape, y.shape, X_test.shape

In [ ]:
lr_model_fit = lr_model.fit(X, y)
final_preds = np.floor(np.expm1(lr_model_fit.predict(X_test)))
print(final_preds)

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission.iloc[:, 1] = final_preds 
print(submission.head())
submission.to_csv("The_first_Regression.csv", index = False)

### 다른 알고리즘 추가

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

lr = LinearRegression() 

dt = DecisionTreeRegressor()

rf = RandomForestRegressor()

gbr = GradientBoostingRegressor()

lst = [dt, rf, gbr]
for model in lst:
  score = cv_rmse(model, n_folds)
  print(score.mean(), score.std())
  rmse_scores[f'model {model}'] = (score.mean(), score.std)


In [ ]:
# 시각화 
fig, ax = plt.subplots(figsize = (10, 6))

ax = sns.pointplot(x = list(rmse_scores.keys()), y=[score for score, _ in rmse_scores.values()], markers=['o'],
                   linestyles = ['-'], ax = ax)
# for i, score in enumerate(rmse_scores.values()):
  # ax.text(i, score[0] + 0.002, '{:.6f}'.format(score[0]), horizontalalignment = 'left',
  #         size = 'large', color = 'black', weight = 'semibold')

ax.set_ylabel('Score (RMSE)', size = 20, labelpad = 12.5)
ax.set_xlabel('Model', size = 20, labelpad = 12.5)
ax.tick_params(axis = 'x', labelsize = 13.5, rotation = 10)
ax.tick_params(axis = 'y', labelsize = 12.5)
ax.set_ylim(0, 0.25)
ax.set_title("RMSE scores of Models without blended predictions", size = 20)
fig.show()

### Blending : 다양한 모델들을 가중치를 다르게 해서 취합

In [ ]:
# Blending : 모델별로 가중치를 다르게 해서 합침
dt_fit = dt.fit(X,y)
rf_fit = rf.fit(X,y)
gbr_fit = gbr.fit(X,y)

def blended_learning_predictions(X):
  blended_score = (0.2 * dt_fit.predict(X)) + (0.4 * rf_fit.predict(X)) + (0.4 * gbr_fit.predict(X))
  return blended_score

In [ ]:
y.values.shape

In [ ]:
blended_learning_predictions(X).shape

In [ ]:
blended_score = rmsle(y, blended_learning_predictions(X))
rmse_scores['blended'] = (blended_score, 0)
print(blended_score) # 위의 스코어는 0.1 아래가 없었던 걸 생각해보자!

In [ ]:
submission.iloc[:, 1]=np.floor(np.expm1(blended_learning_predictions(X_test)))
submission.to_csv('the_2nd_regression.csv', index = False)